# This file runs study 2, varying first batch sizes and no spillover effects

In [1]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import repeat
from scipy import stats
import seaborn as sns
import pandas as pd
import scipy.stats as st

In [2]:
class MAB:
    def __init__(self, probs, karms, nfirst, nEpisodes):
        '''
        meanings for the parameters:
        probs: True success rate for each arm k in {1, ..., K} 
        kArms: Number of arms to choose among
        nsamples: Number of subjects to test at each time step

        '''
        self.n_obs = 4000
        self.probs = probs
        self.K = karms
        self.n = int((self.n_obs-nfirst)/(nEpisodes-1)) if nEpisodes > 1 else 0
        self.T = nEpisodes
        self.first = nfirst
        self.true_win_arm = np.argmax(np.asarray(probs))

    def best_arm(self, s:np.ndarray, asgn):
        draws = 10000
        # + 1 for a Beta(1, 1) prior
        new_alpha = s+1
        new_beta = asgn-s+1
        selection_table = []
        for i in range(self.K):
            theta = np.random.beta(new_alpha[i], new_beta[i], draws)
            selection_table.append(theta)

        winning_arms = np.argmax(selection_table, axis = 0)
        winning_prob = []
        
#         for i in range(self.K):
#             winning_prob.append(winning_arms.count(i)/draws)
        c = Counter(winning_arms)
        winning_prob = np.zeros(self.K)
        for i in range(self.K):
            winning_prob[i] = c[i]/draws

        return winning_prob


    def reset(self):
        # initialization
        success = np.zeros((self.T, self.K))   # record the success cases of each arm at each time step
        assigned = np.zeros((self.T, self.K)) # record the assigned cases of each arm at each time step
        posterior_probs = 0 # estimated success rate
        winning_probs = np.zeros((self.T, self.K)) # probability to be the true best arm
        # rewards = np.zeros(self.T) # record the average rewards per period
        # regrets = np.zeros(self.T) # record the regrets per period
        return (success, assigned, posterior_probs, winning_probs)


    def do_experiment(self, assignments):
        c = Counter(assignments)
        assigned = [c[i] for i in range(self.K)]
        success = [np.random.binomial(assigned[i], self.probs[i]) for i in range(self.K)]
        return assigned, success


    def Thompson_sampling(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        new_assign = np.random.choice(range(self.K), size=self.first)
        assigned[0], success[0]  = self.do_experiment(new_assign)
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        # print(adj_probs)
        if self.T > 1:
            count0=0
            for i in range(1, self.T):              
                new_assign = np.random.choice(range(self.K), size=self.n, p=list(winning_probs[i-1])) # adaptive design
                new_assigned, new_success = self.do_experiment(new_assign)
                assigned[i] = assigned[i-1]+ new_assigned
                success[i] = success[i-1]+ new_success
                winning_probs[i] = self.best_arm(success[i], assigned[i])
                if winning_probs[i-1,0] == 0:
                    count0 += 1
                else:
                    adj_probs += new_success[0]/winning_probs[i-1,0]/self.n
            posterior_probs = adj_probs / (self.T-count0)
        return (success, assigned, posterior_probs, winning_probs)
    


    def static(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        new_assign = np.random.choice(range(self.K), size=self.first).tolist()
        assigned[0], success[0]  = self.do_experiment(new_assign)
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        for i in range(1, self.T):
            new_assign = np.random.choice(range(self.K), size=self.n).tolist() # static design
            new_assigned, new_success = self.do_experiment(new_assign)
            assigned[i] = assigned[i-1]+ new_assigned
            success[i] = success[i-1]+ new_success
            winning_probs[i] = self.best_arm(success[i], assigned[i])
            adj_probs += new_success[0]*self.K/self.n
        posterior_probs = adj_probs / self.T


        return (success, assigned, posterior_probs, winning_probs)
        
    def do_replication(self, times, method):
        '''
        parameters:
        final_regrets -- to record the final regret of each replication, in order to compare efficiency
        final_win_arm -- to record the final win arm selected by two methods, in order to compare the accurarcy
        final_win_probs -- to record the final probability of each arm being the best arm, in order to compare the accurarcy
        final_assignment -- to record the final number of assigned subjects to the true best arm, comparing the exploitition
        cum_rewards -- to record the total rewards of every replication
        '''
        # records of the replication
        final_win_arm = np.zeros(times)
        final_win_probs = np.zeros(shape = (times, self.K))
        final_assignment = np.zeros(times) # only record the true best arm
        estimation = np.zeros(times)
        
        if method == 'TS':
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.Thompson_sampling()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs
        else:
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.static()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs

        ate = np.mean(estimation)
        mse = np.mean(np.square(np.subtract(estimation,self.probs[0])))
        rmse = np.sqrt(mse)
        # rmse = np.std(estimation,0)
#         coverage = best_coverage(estimation, 5000,100,self.probs[0])
        low,high=st.t.interval(0.95, len(estimation)-1, loc=np.mean(estimation), scale=st.sem(estimation))
        win_counts = Counter(final_win_arm)
        estimate = {"best_selected":win_counts[0]/times,"ATE":ate, "RMSE":rmse, "Coverage":(low,high)}
        print(estimate)
        return (final_win_arm, final_win_probs, final_assignment, estimation)

In [3]:
# experiment 1
k = 9 # number of treatments
np.random.seed(99332)
periods = 10
probs = [0.2] + [0.1]*8

In [4]:
first = 100
# exp 1
sim1 = MAB(probs,k,first, periods)
(s1_ts_final_win_arm, s1_ts_final_win_probs, s1_ts_final_assignment, s1_ts_estimation) = sim1.do_replication(5000, "TS")
(s1_st_final_win_arm, s1_st_final_win_probs, s1_st_final_assignment, s1_st_estimation) = sim1.do_replication(5000, "static")

{'best_selected': 1.0, 'ATE': 0.200171319926295, 'RMSE': 0.017748594375876076, 'Coverage': (0.1996792176000399, 0.2006634222525501)}
{'best_selected': 1.0, 'ATE': 0.19968490808314085, 'RMSE': 0.022929924879656553, 'Coverage': (0.1990491771974741, 0.2003206389688076)}


In [5]:
# exp 2
first = 200
sim2 = MAB(probs,k,first, periods)
(s2_ts_final_win_arm, s2_ts_final_win_probs, s2_ts_final_assignment, s2_ts_estimation) = sim2.do_replication(5000, "TS")
(s2_st_final_win_arm, s2_st_final_win_probs, s2_st_final_assignment, s2_st_estimation) = sim2.do_replication(5000, "static")

{'best_selected': 0.9994, 'ATE': 0.20010981906232445, 'RMSE': 0.015701988458658386, 'Coverage': (0.1996744518504463, 0.2005451862742026)}
{'best_selected': 1.0, 'ATE': 0.19981518909952606, 'RMSE': 0.021731622598826696, 'Coverage': (0.19921264597375862, 0.2004177322252935)}


In [6]:
# exp 3
first = 500
sim3 = MAB(probs,k,first, periods)
(s3_ts_final_win_arm, s3_ts_final_win_probs, s3_ts_final_assignment, s3_ts_estimation) = sim3.do_replication(5000, "TS")
(s3_st_final_win_arm, s3_st_final_win_probs, s3_st_final_assignment, s3_st_estimation) = sim3.do_replication(5000, "static")

{'best_selected': 0.9998, 'ATE': 0.19993182611625784, 'RMSE': 0.010197288751971142, 'Coverage': (0.1996490864778007, 0.200214565754715)}
{'best_selected': 1.0, 'ATE': 0.20038993979381445, 'RMSE': 0.02101092578337323, 'Coverage': (0.19980745838282712, 0.20097242120480177)}


In [7]:
# exp 4
first = 1000
sim4 = MAB(probs,k,first, periods)
(s4_ts_final_win_arm, s4_ts_final_win_probs, s4_ts_final_assignment, s4_ts_estimation) = sim4.do_replication(5000, "TS")
(s4_st_final_win_arm, s4_st_final_win_probs, s4_st_final_assignment, s4_st_estimation) = sim4.do_replication(5000, "static")

{'best_selected': 1.0, 'ATE': 0.20001035032699807, 'RMSE': 0.00828078746058701, 'Coverage': (0.1997807444576561, 0.20023995619634005)}
{'best_selected': 1.0, 'ATE': 0.2003749345945946, 'RMSE': 0.022018737379716668, 'Coverage': (0.19976449720418019, 0.20098537198500901)}


In [8]:
# exp 5
first = 2000
sim5 = MAB(probs,k,first, periods)
(s5_ts_final_win_arm, s5_ts_final_win_probs, s5_ts_final_assignment, s5_ts_estimation) = sim5.do_replication(5000, "TS")
(s5_st_final_win_arm, s5_st_final_win_probs, s5_st_final_assignment, s5_st_estimation) = sim5.do_replication(5000, "static")

{'best_selected': 1.0, 'ATE': 0.20020443349025094, 'RMSE': 0.008593433474735328, 'Coverage': (0.19996622596402155, 0.20044264101648032)}
{'best_selected': 1.0, 'ATE': 0.19989571864864866, 'RMSE': 0.026748990071669494, 'Coverage': (0.19915404000108097, 0.20063739729621635)}
